In [ ]:
%matplotlib inline
import sys
sys.path.append("../JSON/")

from models.parameter.base import ParameterModel
from models.command.container import ApptainerDriverCommand
from models.workflow.base import BaseDriverWorkflow
from models.optimizer.base import BaseObjectiveFunction

import torch

In [ ]:
# Create parameter model for kursawe optimization
vector_model = ParameterModel(
    name='Vector',
    data_type='float',  # The vector is a list of floats
    upper_limit=5,      # The upper bound of the optimization problem is 5
    lower_limit=-5,     # The lower bound of the optimization problem is -5
    default=[0,0,0],    # Give all vectors a default of [0,0,0]
    is_list=True        # The vector is a list
)

# Create the parameter class
Vector = vector_model.to_param()

In [ ]:
# Build a command for running f1 of the kursawe microservice (app f1)
f1_kwargs = {
    "name":'run_function_1',
    "parameters": {
        "VECTOR": Vector()                  # The only parameter is the vector
    },
    "uuid": "cluster",                         
    "image_name": 'microservices/kursawe.sif', # The path to the .sif file
    "fn": None,
    "run_app": True,                           # We are running an app of the instance
    "app":'f1',                               # The app we are running is f1 (function 1)
    "start_delay":0,                          # There is no need to have a start delay
    "has_return": True,                        # We are expecting a result to be returned
}
f2_kwargs = {
    "name":'run_function_2',
    "parameters": {
        "VECTOR": Vector()                  # The only parameter is the vector
    },
    "uuid": "cluster",                         
    "image_name": 'microservices/kursawe.sif', # The path to the .sif file
    "fn": None,
    "run_app": True,                           # We are running an app of the instance
    "app":'f2',                               # The app we are running is f1 (function 1)
    "start_delay":0,                          # There is no need to have a start delay
    "has_return": True,                        # We are expecting a result to be returned
}
f1_list = [ApptainerDriverCommand(**f1_kwargs) for i in range(3)]
# Build a command for running f2 of the kursawe microservice (app f2)
f2_list = [ApptainerDriverCommand(**f2_kwargs) for i in range(3)]

In [ ]:
# Start the microservices (instances) linked to each command
for f1 in f1_list: f1.start()
for f2 in f2_list: f2.start()
# for x in f1_list[0]._client.instances(quiet=True): print(x.name) 
for x in f1_list: print(x._instance.name)
for x in f2_list: print(x._instance.name)

In [ ]:
# Build Workflow corresponding to Kursawe function
workflow_list = [BaseDriverWorkflow(
    name="kursawe_workflow",
    commands=[
        f1_list[i],
        f2_list[i],
    ],
) for i in range(len(f1_list))]

exec_kwargs = {
    "list_kwargs": [
        {"VECTOR": [-1,3,4]},
        {"VECTOR": [-1,3,4]},
    ],
    "list_save_vars": [
        {"f1":"f1"},
        {"f2":"f2"}
    ]
}

In [ ]:
workflow_output = [workflow.exec(**exec_kwargs) for workflow in workflow_list]
for elem in workflow_output: print(elem)

In [ ]:
kursawe_obj_fns = [BaseObjectiveFunction(
    name="kursawe_obj_fn",
    workflow= workflow_list[i],
    order_kwargs= [
        {"VECTOR": (0,3)},
        {"VECTOR": (0,3)},
    ],
    list_save_vars=[
        {"f1":"f1"},
        {"f2":"f2"},
    ],
    fitness_criteria=["f1","f2"]
) for i in range(len(workflow_list))]

for fn in kursawe_obj_fns: print()

In [ ]:
fn_output = [obj_fn(torch.Tensor([-1,3,4])) for obj_fn in kursawe_obj_fns]
for elem in fn_output: print(elem)